# television (nettoyage)

cet exercice est originellement proposé ici:

<http://www.xavierdupre.fr/app/ensae_teaching_cs/helpsphinx3/notebooks/td1a_cenonce_session_10.html#exercice-1-creer-un-fichier-excel>

## imports

In [1]:
import numpy as np
import pandas as pd

## la source

l'idée est de se mettre en vraie situation; les données qu'on trouve ici ou là sont souvent très sales !

In [42]:
# sauf que si on le charge: ouh là !

df = pd.read_csv("data/television.txt", sep="\t")
df.columns
df['Unnamed: 9'].unique()


array([nan])

In [3]:
# et en particulier, ceci n'est pas du tout ce qu'on veut

df.shape

(8403, 32)

## survol de ce qu'il faut faire

le TP comporte plusieurs étapes

1. enlever les colonnes pleines de vide; pour fixer les idées, nous nettoyons **les colonnes qui contiennent seulement des n/a ou des 0**
 
   dans le corrigé on va voir deux méthodes
  * rapide
  * manuelle: comment on ferait si le nettoyage devait être fait sur un critère plus spécifique; on verra comment faire sur la base d'une fonction qui, pour une colonne, indique si elle doit être gardée ou pas

1. calculer les valeurs uniques de la colonne `cLT2FREQ`; le texte de l'exercice suggère qu'on doit trouver une poignée de valeurs

1. à ce stade, combien de lignes ont leur `cLT2FREQ` non renseignée ?  
  combien doit-on avoir de lignes si on nettoie sur cette base ?  
  (i.e. si on enlève toutes les lignes qui n'ont pas cette colonne renseignée)
  faites ce nettoyage et vérifiez votre résultat

1. sauver le résultat dans un fichier excel

toujours pour fixer les idées, on doit trouver à la fin une dataframe qui a une forme de `(7386, 4)`

---

## colonnes vides

la première étape donc, consiste à supprimer les colonnes vides

In [4]:
# on recharge pour être sûr
df = pd.read_csv("data/television.txt", sep="\t")
df.shape

(8403, 32)

### la méthode rapide

le mieux c'est d'utiliser `dropna`: écrivez le code ci-dessous et **expliquez votre approche**.

In [14]:
# à vous
df=df.dropna(axis=1, how='all')


In [15]:
# ceci doit afficher True
df.shape == (8403, 4)

True

In [18]:
df.head()


array([0.88942183, 2.31020928, 2.74006977, ..., 2.50118102, 1.38275806,
       0.34333972])

### la méthode pédestre

dans ce cas précis, `dropna` est le mieux bien sûr

maintenant, dans certains cas le critère pour 'oublier' des colonnes peut être moins simple - imaginez par exemple qu'on veuille supprimer toutes les colonnes qui contiennent un certain pourcentage de valeurs parmi `GARBAGE` et `TRASH` et un vrai n/a...

donc voyons comment on peut faire le même nettoyage, mais de manière plus fine

In [ ]:
# on recharge pour être sûr
df = pd.read_csv("data/television.txt", sep="\t")

en deux étapes:

d'abord comment feriez-vous, étant donné le nom d'une colonne, pour savoir si elle est pleine de vide ?

In [71]:
# à vous 
def is_empty_column(df, colname):
    return df[colname].isna().all()==True

In [72]:
# ceci doit afficher True

# on teste
col1 = 'POIDLOG'
not is_empty_column(df, col1)

True

In [73]:
# ceci doit afficher True

col5 = 'Unnamed: 4'

df[col5].unique()
is_empty_column(df, col5)

np.True_

ensuite il ne reste qu'à calculer la liste des colonnes vides, pour la passer à `df.drop()`

In [81]:
# à vous
columns= df.columns
mask=[is_empty_column(df, k) for k in columns]
# calculez la liste des colonnes vides
empty_columns = columns[mask]

# puis utilisez df.drop

df=df.drop(axis=1,labels=empty_columns)

In [82]:
# ceci doit afficher True
df.shape == (8403, 4)

True

Bien sûr on a découpé le problème en deux mais en fait ça peut se récrire en une seule ligne

In [86]:
# en option

# à vous

# récrire tout ceci en une seule passe

df=df.drop(axis=1,labels=df.columns[is_empty_column(df, k) for k in df.columns])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3190631353.py, line 7)

In [87]:
# ceci doit afficher True
df.shape == (8403, 4)

True

## obtenir les valeurs distinctes

comment obtenir les valeurs distinctes de la colonne `cLT2FREQ`

le texte de l'exercice initial nous apprend qu'on ne devrait avoir que 3 valeurs; 
et une inspection visuelle rapide vous le confirme, plus la présence de pas mal de vide dans cette colonne

la méthode la plus simple consiste à utiliser [`Series.unique`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html) qui renvoie le résultat sous la forme d'un `numpy.ndarray`

In [88]:
# à vous
uniques = df['cLT2FREQ'].unique()

In [89]:
uniques

array([ 1.,  2., nan,  3.])

In [90]:
# ceci doit afficher True
uniques.sort()
np.all(uniques[:-1] == np.arange(1, 4)) and np.isnan(uniques[-1])

np.True_

In [99]:
# point de réflexion : pourquoi ceci ne renvoie-t-il pas True ?
uniques.sort()
np.nan== np.nan
np.all(uniques == np.array([1., 2., 3., np.nan])
                           
 # NaN c'est comme pour dire 'pas défini' donc on ne peut pas faire d'opérations avec, sinon on aurait des problèmes 

False

## compter les lignes à nettoyer

on veut maintenant nettoyer les données en enlevant les lignes qui n'ont pas la colonne `cLT2FREQ` renseignée

dans un premier temps on vous demande de calculer le nombre de lignes concernées

In [102]:
# à vous
nb_lines_to_clean = df['cLT2FREQ'].isna().sum()
nb_lines_to_clean


np.int64(1017)

In [103]:
# ceci doit afficher True

nb_lines_to_clean == 1017

np.True_

In [104]:
# ce qui signifie qu'à la fin on doit avoir ce nombre de lignes
8403-1017

7386

In [105]:
# ou encore, plus proprement
expected_lines = len(df) - nb_lines_to_clean
expected_lines

np.int64(7386)

## nettoyage des lignes

### option 1: `df.drop()`

In [107]:
# on recharge à tout hasard (ré-utilisez le code de nettoyage que vous avez réalisé préalablement)
df = pd.read_csv("data/television.txt", sep="\t").dropna(axis=1,how='all')
print(df.shape)

(8403, 4)


remarquez que `df.drop` prend un paramètre optionnel `inplace` qui peut être souvent utile

In [108]:
df.drop?

Signature:
df.drop(
    labels: 'IndexLabel | None' = None,
    *,
    axis: 'Axis' = 0,
    index: 'IndexLabel | None' = None,
    columns: 'IndexLabel | None' = None,
    level: 'Level | None' = None,
    inplace: 'bool' = False,
    errors: 'IgnoreRaise' = 'raise',
) -> 'DataFrame | None'
Docstring:
Drop specified labels from rows or columns.

Remove rows or columns by specifying label names and corresponding
axis, or by directly specifying index or column names. When using a
multi-index, labels on different levels can be removed by specifying
the level. See the :ref:`user guide <advanced.shown_levels>`
for more information about the now unused levels.

Parameters
----------
labels : single label or list-like
    Index or column labels to drop. A tuple will be used as a single
    label and not treated as a list-like.
axis : {0 or 'index', 1 or 'columns'}, default 0
    Whether to drop labels from the index (0 or 'index') or
    columns (1 or 'columns').
index : single label or list

option 1: on peut utiliser `df.drop()`, l'avantage étant qu'on peut faire l'opération en place

In [114]:
df.dropna(inplace=True, subset='cLT2FREQ')


In [115]:
# ceci doit afficher True

# la forme après nettoyage
df.shape == (7386, 4)

True

### option 2: sélection avec un masque et `[]`

In [110]:
# on recharge à tout hasard (ré-utilisez le code de nettoyage que vous avez réalisé préalablement)
df = pd.read_csv("data/television.txt", sep="\t").dropna(axis=1,how='all')
print(df.shape)

(8403, 4)


option 2: il y a plein d'autres façons de faire, on peut aussi utiliser tout simplement un masque

In [124]:
# à vous
df = df[df['cLT2FREQ'].notna()]

In [125]:
# ceci doit afficher True

# la forme après nettoyage
df.shape == (7386, 4)

True

## sauver un fichier excel

In [129]:
!pip install openpyxl

je vous laisse conclure le TP, il s'agit d'enregistrer nos données nettoyées dans un fichier excel

In [130]:
# à vous

df.to_excel( "television.xlsx")
df

,POIDLOG,POIDSF,cLT1FREQ,cLT2FREQ
0,0.889422,4766.865201,2,1.0
1,2.310209,12381.589746,30,1.0
2,2.740070,14685.431344,6,2.0
3,1.775545,9516.049939,1,1.0
4,0.732512,3925.907588,3,1.0
...,...,...,...,...
8397,0.502091,2690.961176,3,1.0
8398,0.306852,1644.574141,6,1.0
8399,2.501181,13405.104689,6,1.0
8400,1.382758,7410.905653,1,1.0


je vous laisse éventuellement vérifier votre code en rechargeant sous excel le fichier produit

![](media/television.png)

***